## Reading in data available through the Planteome database
The purpose of this notebook is to read in and do a preliminary survey of the data related to text and ontology annotations of text that is available through TAIR. The datasets need to be organized and also restructured into a standard format that will allow it be combined with datasets from other resources. This notebook takes the following input files that were downloaded from the TAIR and produces a set of files that have standard columns, which are listed and described below.

### Files read
```
plant-data/databases/planteome/biological_process.txt
plant-data/databases/planteome/cellular_component.txt
plant-data/databases/planteome/molecular_function.txt
plant-data/databases/planteome/plant_anatomical_entity.txt
plant-data/databases/planteome/plant_structure_development_stage.txt
plant-data/databases/planteome/quality.txt
```

### Files created
```
plant-data/reshaped_data/planteome_curated_annotations.csv
```

### Columns in the created files
* **species**: A string indicating what species the gene is in, currently uses the 3-letter codes from the KEGG database.
* **unique_gene_identifiers**: Pipe delimited list of gene identifers, names, models, etc which must uniquely refer to this gene.
* **other_gene_identifiers**: Pipe delimited list of other identifers, names, aliases, synonyms for the gene, which may but do not have to uniquely refer to it.
* **gene_models**: Pipe delimited list of gene model names that map to this gene.
* **descriptions**: A free text field for any descriptions of phenotyes associated with this gene.
* **annotations**: Pipe delimited list of gene ontology term identifiers.
* **sources**: Pipe delimited list of strings that indicate where this data comes from such as database names.

In [1]:
import sys
import os
import re
import warnings
import pandas as pd
import numpy as np
import itertools
import re

sys.path.append("../utils")
from constants import EVIDENCE_CODES

OUTPUT_DIR = "../reshaped_data"
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Creating a list of lambdas for finding gene model strings.
gene_model_patterns = []
gene_model_patterns.append(re.compile("grmzm.+"))
gene_model_patterns.append(re.compile("zm[0-9]+d[0-9]+"))
gene_model_patterns.append(re.compile("solyc[0-9]+g[0-9]+"))
gene_model_patterns.append(re.compile("at[0-9]{1}g[0-9]+"))
gene_model_patterns.append(re.compile("medtr[0-9]{1}g[0-9]+"))
gene_model_patterns.append(re.compile("os[0-9]+g[0-9]+"))
gene_model_patterns.append(re.compile("loc_os[0-9]+g[0-9]+"))
is_gene_model = lambda s: any([bool(pattern.match(s.lower())) for pattern in gene_model_patterns])

In [3]:
# Columns that should be in the final reshaped files.
reshaped_columns = ["species", 
 "unique_gene_identifiers", 
 "other_gene_identifiers", 
 "gene_models", 
 "descriptions", 
 "annotations", 
 "sources"]

# The files that were obtained through the Planteome browser queries.
planteome_annotation_filepaths = [
    "../databases/planteome/biological_process.txt",
    "../databases/planteome/cellular_component.txt",
    "../databases/planteome/molecular_function.txt",
    "../databases/planteome/plant_anatomical_entity.txt",
    "../databases/planteome/plant_structure_development_stage.txt",
    "../databases/planteome/quality.txt"
]
# The fields that were included for each of those downloaded files.
columns = [
    "bioentity",
    "bioentity_name",
    "type",
    "bioentity_label",
    "annotation_class",
    "aspect",
    "annotation_extension_json",
    "taxon",
    "evidence_type",
    "evidence_with",
    "reference",
    "assigned_by"
]
# Read in all the files and stack rows, because they all have the same formatting and fields.
dfs = [pd.read_csv(path, sep="\t", names=columns) for path in planteome_annotation_filepaths]
df = pd.concat(dfs)
df.head(20)

,bioentity,bioentity_name,type,bioentity_label,annotation_class,aspect,annotation_extension_json,taxon,evidence_type,evidence_with,reference,assigned_by
0,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
1,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0016032,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
2,GR_gene:GR:0116029,LOC_Os07g41220,gene,LOC_Os07g41220,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
3,GR_gene:GR:0136509,LOC_Os05g45110,gene,LOC_Os05g45110,GO:0008152,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
4,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0006333,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
5,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
6,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
7,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
8,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
9,GR_gene:GR:0136500,LOC_Os05g43960,gene,LOC_Os05g43960,GO:0006355,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene


In [4]:
df.tail(20)

,bioentity,bioentity_name,type,bioentity_label,annotation_class,aspect,annotation_extension_json,taxon,evidence_type,evidence_with,reference,assigned_by
3394,SGN_gene:1418,Solyc03g083910,gene,sucr,TO:0000328,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3395,SGN_gene:1345,HQ317907,gene,sit,TO:0000207,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3396,SGN_gene:1345,HQ317907,gene,sit,TO:0000541,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3397,SGN_gene:1345,HQ317907,gene,sit,TO:0000652,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3398,SGN_gene:1345,HQ317907,gene,sit,TO:0002667,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3399,SGN_gene:1296,Solyc05g012020,gene,mc,TO:0000373,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3400,SGN_gene:1296,Solyc05g012020,gene,mc,TO:0000864,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3401,SGN_gene:1296,Solyc05g012020,gene,mc,TO:0002604,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3402,SGN_gene:1296,Solyc05g012020,gene,mc,TO:0002617,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3403,SGN_gene:1296,Solyc05g012020,gene,mc,TO:0002633,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson


In [5]:
# How many of each evidence type are there in this combined?
code_quantities = {c:len([x for x in df["evidence_type"] if EVIDENCE_CODES[x] in c]) for c in list(set(EVIDENCE_CODES.values()))}
for k,v in code_quantities.items():
    print("{:25}{:8}".format(k,v))

experimental               168565
electronic_annotation           2
curator_statement            1118
author_statement             9980
phylogenetics                   0
computational                 138
high_throughput                 0


In [6]:
df.shape

(179803, 12)

In [7]:
# Retain just the annotations that we're considering high confidence.
high_confidence_categories = ["experimental","author_statement","curator_statement"]
df["high_confidence"] = df["evidence_type"].apply(lambda x: EVIDENCE_CODES[x] in high_confidence_categories)
df = df[df["high_confidence"]]
df.shape

(179663, 13)

In [8]:
# Taxon IDs are given in the file but we need to use the naming scheme used across files to make it compatible.
ncbi_taxon_ids = ["3702","3847","3880","4081","4530","4577"]
species_name_strings = ["ath","gmx","osa","mtr","sly","zma"]
mapping = dict(zip(ncbi_taxon_ids,species_name_strings))
df["species"] = df["taxon"].map(lambda x: mapping.get(x[-4:],None))
df.dropna(subset=["species"], axis=0, inplace=True)
pd.unique(df["species"])

array(['sly', 'mtr', 'zma', 'ath', 'gmx', 'osa'], dtype=object)

In [9]:
df.shape

(176002, 14)

In [10]:
df.head(20)

,bioentity,bioentity_name,type,bioentity_label,annotation_class,aspect,annotation_extension_json,taxon,evidence_type,evidence_with,reference,assigned_by,high_confidence,species
0,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly
1,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0016032,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly
2,GR_gene:GR:0116029,LOC_Os07g41220,gene,LOC_Os07g41220,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly
3,GR_gene:GR:0136509,LOC_Os05g45110,gene,LOC_Os05g45110,GO:0008152,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly
4,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0006333,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly
5,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly
6,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly
7,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly
8,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly
9,GR_gene:GR:0136500,LOC_Os05g43960,gene,LOC_Os05g43960,GO:0006355,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly


In [11]:
# Formatting the gene identifier columns to be the same as the other files.
df["unique_gene_identifiers"] = df.apply(lambda row: "{}|{}".format(row["bioentity_name"],row["bioentity_label"]), axis=1)
df["other_gene_identifiers"] = ""
df.head(20)

,bioentity,bioentity_name,type,bioentity_label,annotation_class,aspect,annotation_extension_json,taxon,evidence_type,evidence_with,reference,assigned_by,high_confidence,species,unique_gene_identifiers,other_gene_identifiers
0,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,LOC_Os06g40240|LOC_Os06g40240,
1,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0016032,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,LOC_Os06g40240|LOC_Os06g40240,
2,GR_gene:GR:0116029,LOC_Os07g41220,gene,LOC_Os07g41220,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,LOC_Os07g41220|LOC_Os07g41220,
3,GR_gene:GR:0136509,LOC_Os05g45110,gene,LOC_Os05g45110,GO:0008152,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,LOC_Os05g45110|LOC_Os05g45110,
4,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0006333,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,LOC_Os08g30200|LOC_Os08g30200,
5,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,LOC_Os08g30200|LOC_Os08g30200,
6,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,LOC_Os12g26444|LOC_Os12g26444,
7,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,LOC_Os12g26444|LOC_Os12g26444,
8,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,LOC_Os12g26444|LOC_Os12g26444,
9,GR_gene:GR:0136500,LOC_Os05g43960,gene,LOC_Os05g43960,GO:0006355,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,LOC_Os05g43960|LOC_Os05g43960,


In [12]:
# Other columns that are needed.
df["gene_models"] = df["unique_gene_identifiers"].map(lambda x: "".join([s for s in x.split("|") if is_gene_model(s)]))
df["descriptions"] = ""
df["annotations"] = df["annotation_class"]
df["sources"] = "Planteome"

In [13]:
df["ontology"] = df["annotation_class"].map(lambda x: x.split(":")[0])
pd.unique(df["ontology"])
df = df[df["ontology"].isin(["GO","PO","PATO"])]
df.shape

(172590, 21)

In [14]:
df = df[reshaped_columns]
df.head(30)

,species,unique_gene_identifiers,other_gene_identifiers,gene_models,descriptions,annotations,sources
0,sly,LOC_Os06g40240|LOC_Os06g40240,,LOC_Os06g40240LOC_Os06g40240,,GO:0006278,Planteome
1,sly,LOC_Os06g40240|LOC_Os06g40240,,LOC_Os06g40240LOC_Os06g40240,,GO:0016032,Planteome
2,sly,LOC_Os07g41220|LOC_Os07g41220,,LOC_Os07g41220LOC_Os07g41220,,GO:0006508,Planteome
3,sly,LOC_Os05g45110|LOC_Os05g45110,,LOC_Os05g45110LOC_Os05g45110,,GO:0008152,Planteome
4,sly,LOC_Os08g30200|LOC_Os08g30200,,LOC_Os08g30200LOC_Os08g30200,,GO:0006333,Planteome
5,sly,LOC_Os08g30200|LOC_Os08g30200,,LOC_Os08g30200LOC_Os08g30200,,GO:0015074,Planteome
6,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0006278,Planteome
7,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0006508,Planteome
8,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0015074,Planteome
9,sly,LOC_Os05g43960|LOC_Os05g43960,,LOC_Os05g43960LOC_Os05g43960,,GO:0006355,Planteome


In [15]:
# Outputting the dataset of annotations to a csv file.
path = os.path.join(OUTPUT_DIR,"planteome_curated_annotations.csv")
df.to_csv(path, index=False)
df.head(30)

,species,unique_gene_identifiers,other_gene_identifiers,gene_models,descriptions,annotations,sources
0,sly,LOC_Os06g40240|LOC_Os06g40240,,LOC_Os06g40240LOC_Os06g40240,,GO:0006278,Planteome
1,sly,LOC_Os06g40240|LOC_Os06g40240,,LOC_Os06g40240LOC_Os06g40240,,GO:0016032,Planteome
2,sly,LOC_Os07g41220|LOC_Os07g41220,,LOC_Os07g41220LOC_Os07g41220,,GO:0006508,Planteome
3,sly,LOC_Os05g45110|LOC_Os05g45110,,LOC_Os05g45110LOC_Os05g45110,,GO:0008152,Planteome
4,sly,LOC_Os08g30200|LOC_Os08g30200,,LOC_Os08g30200LOC_Os08g30200,,GO:0006333,Planteome
5,sly,LOC_Os08g30200|LOC_Os08g30200,,LOC_Os08g30200LOC_Os08g30200,,GO:0015074,Planteome
6,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0006278,Planteome
7,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0006508,Planteome
8,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0015074,Planteome
9,sly,LOC_Os05g43960|LOC_Os05g43960,,LOC_Os05g43960LOC_Os05g43960,,GO:0006355,Planteome
